In [2]:
import numpy as np
import sys
sys.path.append(
    'C:\\Users\\drago\\university\\23.1 semester\\DL\\DLmodels-tensorflow')
print(sys.path)

from tensorflow import keras
from  tensorflow.keras.optimizers import Adam 
from readData import my_data_reader
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add

import os
import matplotlib.pyplot as plt
import numpy as np
import math
import glob
from PIL import Image

['c:\\Users\\drago\\university\\23.1 semester\\DL\\DLmodels-tensorflow\\examples\\assignment', 'c:\\Users\\drago\\anaconda3\\envs\\py36tf131\\python36.zip', 'c:\\Users\\drago\\anaconda3\\envs\\py36tf131\\DLLs', 'c:\\Users\\drago\\anaconda3\\envs\\py36tf131\\lib', 'c:\\Users\\drago\\anaconda3\\envs\\py36tf131', '', 'c:\\Users\\drago\\anaconda3\\envs\\py36tf131\\lib\\site-packages', 'c:\\Users\\drago\\anaconda3\\envs\\py36tf131\\lib\\site-packages\\win32', 'c:\\Users\\drago\\anaconda3\\envs\\py36tf131\\lib\\site-packages\\win32\\lib', 'c:\\Users\\drago\\anaconda3\\envs\\py36tf131\\lib\\site-packages\\Pythonwin', 'c:\\Users\\drago\\anaconda3\\envs\\py36tf131\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\drago\\.ipython', 'C:\\Users\\drago\\university\\23.1 semester\\DL\\DLmodels-tensorflow', 'C:\\Users\\drago\\university\\23.1 semester\\DL\\DLmodels-tensorflow']


In [3]:
x_train = []
x_test = []
y_train = []
y_test = []


img_size = 150

train_folder_path = glob.glob("../../data/medical/train/*")
print("Read folder path : " + str(train_folder_path))
test_folder_path = glob.glob("../../data/medical/test/*")
print("Read folder path : " + str(test_folder_path))


folder_path = train_folder_path + test_folder_path

numFiles = [0] * len(folder_path)  # numFiles 리스트 초기화
file_path = []
for i, path in enumerate(folder_path):
        temp_path = glob.glob(path + "/*")  # 폴더 내 모든 파일 경로 가져오기
        numFiles[i] = len(temp_path)
        file_path += temp_path

print("size of data set : " + str(len(file_path)))
print("size of each label : " + str(numFiles))

data = []
for i, path in enumerate(file_path):
    img = Image.open(path)
    if i == 0 : print("img resize " + str(img.size[1]) + " to " + str(img_size))
    img = img.resize((img_size, img_size))
    img = np.asarray(img)
    # print(img.shape)

    if i <= 80:  # 파일이 속한 폴더의 인덱스 계산
        img_label = 0            
        
    elif i <= 160 and i > 80:  # 파일이 속한 폴더의 인덱스 계산
        img_label = 1            
        
    elif i <= 220 and i > 160:  # 파일이 속한 폴더의 인덱스 계산
        img_label = 0            
        
    elif i <= 280 and i > 220:  # 파일이 속한 폴더의 인덱스 계산
        img_label = 1            
        
    else :
        img_label = 0      
        
    if i % 100 == 0:
        print("labeling data set : " + str(i) + "/" + str(len(file_path)))
    data.append((img, img_label))
    # break
print("labeling data done : " + str(len(file_path)) + "/" + str(len(file_path)))

Read folder path : ['../../data/medical/train\\ad', '../../data/medical/train\\normal']
Read folder path : ['../../data/medical/test\\ad', '../../data/medical/test\\normal']
size of data set : 280
size of each label : [80, 80, 60, 60]
img resize 150 to 150
labeling data set : 0/280
labeling data set : 100/280
labeling data set : 200/280
labeling data done : 280/280


In [4]:
data = np.array(data)
a = [row[1] for row in data]
np.unique(a)

c:\Users\drago\anaconda3\envs\py36tf131\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


array([0, 1])

In [5]:
data[0]

array([array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8),
       0], dtype=object)

In [64]:
train_data, test_data= np.split(data, [160])
print(train_data.shape)
print(train_data[0][0].shape)
print(test_data.shape)
print(test_data[0][0].shape)

(160, 2)
(150, 150, 3)
(120, 2)
(150, 150, 3)


In [65]:

augmentedset = []
for i in range(160):
    x = np.array(train_data[i][0])
    y = train_data[i][1]
    newdata = np.fliplr(x)
    augmentedset.append((newdata, y))


augmentedset = np.array(augmentedset)
print(augmentedset.shape)
print(augmentedset[0][0].shape)
train_data  = np.concatenate((train_data, augmentedset), axis=0)
print(train_data.shape)
print(train_data[0][0].shape)


(160, 2)
(150, 150, 3)
(320, 2)
(150, 150, 3)


c:\Users\drago\anaconda3\envs\py36tf131\lib\site-packages\ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


In [66]:
train_target = [row[1] for row in train_data]
train_data2 = [row[0] for row in train_data]

test_target = [row[1] for row in test_data]
test_data2 = [row[0] for row in test_data]

train_data2[0].shape

(150, 150, 3)

In [67]:
print(np.unique(train_target))
print(np.unique(test_target))

[0 1]
[0 1]


In [68]:
x_train = np.array(train_data2) / 255.0
x_test = np.array(test_data2) / 255.0

y_train = np.array(train_target) / 1.0
y_test = np.array(test_target) / 1.0

In [72]:
print(np.unique(y_test))
print(np.unique(y_train))

[0. 1.]
[0. 1.]


In [75]:
x_train.shape , x_test.shape , y_test.shape , y_train.shape

((320, 150, 150, 3), (120, 150, 150, 3), (120,), (320,))

In [82]:
from tensorflow import keras
from  tensorflow.keras.optimizers import Adam 

model = keras.Sequential([
    # 이걸 수정해보자
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation ='ReLU' ),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation ='ReLU' ),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation ='ReLU' ),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation ='ReLU' ),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(2000, activation = 'ReLU'),
    #classifing 2
    keras.layers.Dense(2, activation = 'softmax')
])

In [84]:
model.compile(optimizer = Adam(learning_rate= 0.001), metrics=['accuracy'],loss = 'sparse_categorical_crossentropy')
print("model definded\n\n", model.summary)

print("\n\n************ TRAINING START ************ ")
print(y_train.dtype)
history = model.fit(x_train, y_train, epochs=10,
                    validation_data=(x_test, y_test))

res = model.evaluate(x_test,y_test,verbose = 0 )
print("acc is : ", res[1]* 100)

model definded

 <bound method Model.summary of <keras.engine.sequential.Sequential object at 0x000001AD8723FD68>>


************ TRAINING START ************ 
float64
Epoch 1/10
10/10 [==============================] - 7s 661ms/step - loss: 0.4954 - accuracy: 0.7594 - val_loss: 0.4719 - val_accuracy: 0.7750
Epoch 2/10
10/10 [==============================] - 7s 682ms/step - loss: 0.2332 - accuracy: 0.9250 - val_loss: 0.2042 - val_accuracy: 0.9500
Epoch 3/10
10/10 [==============================] - 6s 617ms/step - loss: 0.1703 - accuracy: 0.9438 - val_loss: 0.2066 - val_accuracy: 0.9500
Epoch 4/10
10/10 [==============================] - 7s 714ms/step - loss: 0.1857 - accuracy: 0.9281 - val_loss: 0.2688 - val_accuracy: 0.9083
Epoch 5/10
10/10 [==============================] - 8s 811ms/step - loss: 0.1114 - accuracy: 0.9594 - val_loss: 0.1865 - val_accuracy: 0.9500
Epoch 6/10
10/10 [==============================] - 8s 790ms/step - loss: 0.1041 - accuracy: 0.9625 - val_loss: 0.1645 - va